You are given an integer n, which represents the numbers from 1 to n, and a string that is a concatenation of all numbers in this range except one missing number.  

Your task is to find the missing number.

For example, if n = 5 and the string is '1234', the missing number is 5.

---

Follow-up: The numbers in the string may appear in any order. For example, if n = 13, the string can be '1310923....'. 

### Solution 1, Ordered String of Numbers, Greedy Linear Scan with Sequential Prefix Matching
Solution description
- Time Complexity: O(N*D) ~ effectively O(n log n).
  - where d is the number of digits in n
- Space Complexity: O(log N)
  - just storing the current number as a string.

In [ ]:
def find_missing_number(n: int, s: str) -> int:
    idx = 0
    
    for num in range(1, n + 1):
        num_str = str(num)
        if s[idx:idx + len(num_str)] == num_str:
            idx += len(num_str)
        else:
            return num
    return -1  # Should never reach here if input is valid

# Test Case
n_val = 13
ordered_str = "123456789101113" # Missing 12
print(f"Missing: {find_missing_number(n_val, ordered_str)}")

n_val = 5
ordered_str = "1234" # Missing 5
print(f"Missing: {find_missing_number(n_val, ordered_str)}")

### Solution 1, Ordered String of Numbers, Backtracking approach
Solution description
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
def find_missing_ordered(n, s):
    def backtrack(index, expected_num, skipped):
        # Base Case: If we reached the end of the string
        if index == len(s):
            # Valid if we skipped exactly one number OR if the last number was skipped
            return skipped or expected_num == n
        
        # If we've already surpassed n, this path is invalid
        if expected_num > n:
            return None

        # Try to match the current expected number
        num_str = str(expected_num)
        if s.startswith(num_str, index):
            # Match found, move to the next number in sequence
            res = backtrack(index + len(num_str), expected_num + 1, skipped)
            if res: return res

        # If no match, try skipping this number (only if we haven't skipped one yet)
        if not skipped:
            res = backtrack(index, expected_num + 1, True)
            if res: return expected_num
            
        return None

    return backtrack(0, 1, False)

# Test Case
n_val = 13
# '123456789101113' (Missing 12)
ordered_str = "123456789101113" 
print(f"The missing number is: {find_missing_ordered(n_val, ordered_str)}")

### Solution 2, Unordered String of Numbers, Digit Frequency Analysis approach

The Critical Flaw — Anagram Ambiguity:
The digit-count diff only tells you which digits are missing, not which number. This breaks when multiple candidates share the same digits:

n = 99, missing number is 13
But Counter('13') == Counter('31'), so if 31 is also a candidate, it's ambiguous
find_missing_number_refined(99, s_missing_13)  # Might return 13 or 31 — wrong!

More concretely, if the diff is {'1': 1, '3': 1}, both 13 and 31 match, and the function returns whichever it hits first in the loop — which may be incorrect.
When it's guaranteed to work:

When n < 10 (all single-digit numbers, no anagram issue)
When the digit diff happens to correspond to only one valid number in range

Solution description
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
from collections import Counter

def find_missing_number_refined(n, concatenated_str):
    # 1. Calculate what digits SHOULD be there
    expected_counts = Counter()
    for i in range(1, n + 1):
        expected_counts.update(str(i))
            
    # 2. Calculate what IS there
    actual_counts = Counter(concatenated_str)
    
    # 3. Find the "Digit Debt" (the difference)
    diff_counts = expected_counts - actual_counts
    
    # 4. Find which number from 1 to n matches this exact "Debt"
    for i in range(1, n + 1):
        if Counter(str(i)) == diff_counts:
            return i
            
    return "No missing number found"

# Corrected Example
n_val = 13
# Let's manually build a string for 1..13 missing the number '12'
full_sequence = "".join(str(i) for i in range(1, 14) if i != 12)
# Shuffling is irrelevant to the Counter logic
shuffled_str = "".join(sorted(full_sequence, reverse=True)) 

print(f"String: {shuffled_str}")
print(f"The missing number is: {find_missing_number_refined(n_val, shuffled_str)}")

### Solution 2.2, Fixed Digit Frequency Differencing approach
Solution description
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
from collections import Counter

def find_missing_number(n: int, s: str) -> int:
    # Step 1: Find candidates via digit frequency differencing
    expected_counts = Counter()
    for i in range(1, n + 1):
        expected_counts.update(str(i))

    actual_counts = Counter(s)
    diff_counts = expected_counts - actual_counts

    candidates = [i for i in range(1, n + 1) if Counter(str(i)) == diff_counts]

    # Step 2: If only one candidate, return it directly
    if len(candidates) == 1:
        return candidates[0]

    # Step 3: Otherwise, use backtracking to find the true missing number
    all_numbers = set(range(1, n + 1))

    def backtrack(pos: int, remaining: set) -> int | None:
        if pos == len(s):
            if len(remaining) == 1:
                return next(iter(remaining))
            return None

        for length in range(1, len(s) - pos + 1):
            token = s[pos:pos + length]
            if token[0] == '0':
                break
            num = int(token)
            if num in remaining:
                remaining.remove(num)
                result = backtrack(pos + length, remaining)
                if result is not None:
                    return result
                remaining.add(num)

        return None

    # Only backtrack with candidates as the "suspicious" numbers
    # We know the missing number must be one of the candidates
    non_candidates = all_numbers - set(candidates)
    reduced_set = all_numbers.copy()

    def backtrack_refined(pos: int, remaining: set) -> int | None:
        if pos == len(s):
            leftover = remaining & set(candidates)
            if len(leftover) == 1:
                return next(iter(leftover))
            return None

        for length in range(1, len(s) - pos + 1):
            token = s[pos:pos + length]
            if token[0] == '0':
                break
            num = int(token)
            if num in remaining:
                remaining.remove(num)
                result = backtrack_refined(pos + length, remaining)
                if result is not None:
                    return result
                remaining.add(num)

        return None

    return backtrack_refined(0, reduced_set)


# --- Tests ---
if __name__ == "__main__":
    test_cases = [
        (5,  "4152",                              3),
        (5,  "5132",                              4),
        (5,  "1234",                              5),
        (13, "1310923456781112",                  2),  # adjusted to be valid
        (9,  "987654321",                         None),  # no missing (sanity)
    ]

    for n, s, expected in test_cases:
        result = find_missing_number(n, s)
        status = "✓" if result == expected else f"✗ (got {result})"
        print(f"n={n}, s='{s}' → missing: {result} {status}")